# Round#1

- Task: Fetch LLM around 20 benign and attack entries and ask the next input is
attack or benign.
- Result: I tested with 20 benign and attack entries. It can predict attack 
entries with higher recall. However, precision of benign entries is bit lower.
Token count reached upto 11000 with commands + entries.


In [65]:
from openai import OpenAI
import tiktoken
import os
import dotenv
import pandas as pd

dotenv.load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=API_KEY)
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# read csv file
df_normal = pd.read_csv("X_train_normal.csv")
df_attack = pd.read_csv("X_train_attack.csv")

messages = [
    {"role": "system", "content": "User gives 10 samples of normal (benign) and attack iot network data."+
     "When user ask to predict the label given a sample, analyze the samples and check whether the sample is benign or not."+
     "Output the label 'attack' or 'benign', nothing else."+
     f"The samples has following fields respectively {df_normal.columns.tolist()}."},
    
    {"role": "user", "content": str(df_attack.iloc[1].values.tolist()) + "> 'attack'"},
    {"role": "user", "content": str(df_attack.iloc[2].values.tolist()) + "> 'attack'"},
    {"role": "user", "content": str(df_attack.iloc[3].values.tolist()) + "> 'attack'"},
    {"role": "user", "content": str(df_attack.iloc[4].values.tolist()) + "> 'attack'"},
    {"role": "user", "content": str(df_attack.iloc[5].values.tolist()) + "> 'attack'"},
    {"role": "user", "content": str(df_attack.iloc[6].values.tolist()) + "> 'attack'"},
    {"role": "user", "content": str(df_attack.iloc[7].values.tolist()) + "> 'attack'"},
    {"role": "user", "content": str(df_attack.iloc[8].values.tolist()) + "> 'attack'"},
    {"role": "user", "content": str(df_attack.iloc[9].values.tolist()) + "> 'attack'"},
    {"role": "user", "content": str(df_normal.iloc[1].values.tolist()) + "> 'benign'"},
    {"role": "user", "content": str(df_normal.iloc[2].values.tolist()) + "> 'benign'"},
    {"role": "user", "content": str(df_normal.iloc[3].values.tolist()) + "> 'benign'"},
    {"role": "user", "content": str(df_normal.iloc[4].values.tolist()) + "> 'benign'"},
    {"role": "user", "content": str(df_normal.iloc[5].values.tolist()) + "> 'benign'"},
    {"role": "user", "content": str(df_normal.iloc[6].values.tolist()) + "> 'benign'"},
    {"role": "user", "content": str(df_normal.iloc[7].values.tolist()) + "> 'benign'"},
    {"role": "user", "content": str(df_normal.iloc[8].values.tolist()) + "> 'benign'"},
    {"role": "user", "content": str(df_normal.iloc[9].values.tolist()) + "> 'benign'"},
    {"role": "user", "content": "Predict this entry is attack or benign: "+str(df_normal.iloc[15].values.tolist())}]
    # {"role": "user", "content": "Predict this entry is attack or benign: "+str(df_attack.iloc[19].values.tolist())}]

print("Messages:", messages)
num_tokens = len(encoding.encode(str(messages)))
print("Num tokens:", num_tokens)
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages
)
response = completion.choices[0].message.content.strip()
print(response)


Messages: [{'role': 'system', 'content': "User gives 10 samples of normal (benign) and attack iot network data.When user ask to predict the label given a sample, analyze the samples and check whether the sample is benign or not.Output the label 'attack' or 'benign', nothing else.The samples has following fields respectively ['frame.time', 'ip.src_host', 'ip.dst_host', 'arp.dst.proto_ipv4', 'arp.opcode', 'arp.hw.size', 'arp.src.proto_ipv4', 'icmp.checksum', 'icmp.seq_le', 'icmp.transmit_timestamp', 'icmp.unused', 'http.file_data', 'http.content_length', 'http.request.uri.query', 'http.request.method', 'http.referer', 'http.request.full_uri', 'http.request.version', 'http.response', 'http.tls_port', 'tcp.ack', 'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack', 'tcp.dstport', 'tcp.flags', 'tcp.flags.ack', 'tcp.len', 'tcp.options', 'tcp.payload', 'tcp.seq', 'tcp.srcport', 'udp.port', 'udp.stream', 'udp.time_delta', 'dns

# Round#2

- Task: Use langchain to do the same task.
- Result: Bit improvement can be seen for benign entries but 1 attack entry is predicted false.

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
import os
import dotenv
import pandas as pd

dotenv.load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(openai_api_key=API_KEY, temperature=0.0)

# read csv file
df_normal = pd.read_csv("X_train_normal.csv")
df_attack = pd.read_csv("X_train_attack.csv")

prompt = ChatPromptTemplate.from_messages([
    ("system", f"You are intelligent log analyzer. User gives you samples of benign and attack iot network data. When user ask to predict the label given a sample, analyze the samples and check whether the sample is benign or not. Output the label 'attack' or 'benign', nothing else. The samples has following fields respectively {df_normal.columns.tolist()}."),
    ("user", str(df_attack.iloc[1].values.tolist()) + "> 'attack'"),
    ("user", str(df_attack.iloc[2].values.tolist()) + "> 'attack'"),
    ("user", str(df_attack.iloc[3].values.tolist()) + "> 'attack'"),
    ("user", str(df_attack.iloc[4].values.tolist()) + "> 'attack'"),
    ("user", str(df_attack.iloc[5].values.tolist()) + "> 'attack'"),
    ("user", str(df_attack.iloc[6].values.tolist()) + "> 'attack'"),
    ("user", str(df_attack.iloc[7].values.tolist()) + "> 'attack'"),
    ("user", str(df_attack.iloc[8].values.tolist()) + "> 'attack'"),
    ("user", str(df_attack.iloc[9].values.tolist()) + "> 'attack'"),
    ("user", str(df_normal.iloc[1].values.tolist()) + "> 'benign'"),
    ("user", str(df_normal.iloc[2].values.tolist()) + "> 'benign'"),
    ("user", str(df_normal.iloc[3].values.tolist()) + "> 'benign'"),
    ("user", str(df_normal.iloc[4].values.tolist()) + "> 'benign'"),
    ("user", str(df_normal.iloc[5].values.tolist()) + "> 'benign'"),
    ("user", str(df_normal.iloc[6].values.tolist()) + "> 'benign'"),
    ("user", str(df_normal.iloc[7].values.tolist()) + "> 'benign'"),
    ("user", str(df_normal.iloc[8].values.tolist()) + "> 'benign'"),
    ("user", str(df_normal.iloc[9].values.tolist()) + "> 'benign'"),
    ("user", "{input}")
])

chain = prompt | llm 
for i in range(10, 20):
    print(chain.invoke({"input": "Predict this entry is attack or benign: " + str(df_normal.iloc[i].values.tolist())}))
for i in range(10, 20):
    print(chain.invoke({"input": "Predict this entry is attack or benign: " + str(df_attack.iloc[i].values.tolist())}))



content='The dataset appears to be related to network traffic or cybersecurity, as the samples contain information such as IP addresses, timestamps, and network activity.' response_metadata={'finish_reason': 'stop', 'logprobs': None}


# Round#3

- Task: